In [1]:
from ssri_interactions.interactions.loaders import BaseShockSlowInteractionsLoader
from ssri_interactions.interactions.preprocessors import InteractionsPreprocessor
from ssri_interactions.interactions.pairwise import PairwiseCorr
from ssri_interactions.interactions.graph_clustering import df_to_graph
from ssri_interactions.config import Config, ExperimentInfo
import numpy as np
from ssri_interactions.interactions.graph import GraphAttributes, NodeAttributes
import pandas as pd
import seaborn as sns
from ssri_interactions.io import load_derived_generic, load_distances
from ssri_interactions.transforms.graph import GraphTransformer
import matplotlib.pyplot as plt
from ssri_interactions.plots import PAL_GREY_BLACK
from scipy.stats import chi2_contingency
from tqdm import tqdm
from scipy.stats import wilcoxon
from ssri_interactions.stats import mannwhitneyu_plusplus
import pingouin as pg
from scipy.stats import spearmanr

fig_dir = Config.fig_dir
sns.set_theme(style="ticks", context="paper")

c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
def load_responders():
    slow_responders_shock = (
        load_derived_generic("slow_ts_foot_shock_unit_responders_pre_to_shock.csv")
        [["neuron_id", "response"]]
        .rename(columns={"response": "response_fs_slow"})
    )
    fast_responders_first_window = (
        load_derived_generic("fast_fs_foot_shock_unit_responders.csv")
        [["neuron_id", "fs_fast_response"]]
    )
    fast_responders_second_window = (
        load_derived_generic("fast_fs_foot_shock_unit_responders_second_window.csv")
        [["neuron_id", "response_second_window"]]
    )
    neuron_types = load_derived_generic("neuron_types.csv")[["neuron_id", "neuron_type", "session_name", "group"]]
    df_responders = (
        slow_responders_shock
        .merge(fast_responders_first_window, how="outer")
        .merge(fast_responders_second_window, how="outer")
        .merge(neuron_types, how="left")
    )
    return df_responders


def get_graph_stats(session_name, bin_width, graph_transformer, block):
        graph_attrs = GraphAttributes(distance_from_edge="bounded")
        node_attrs = NodeAttributes()
        loader = BaseShockSlowInteractionsLoader(
                        session_name=session_name,
                        bin_width=bin_width,
                        block=block,
                    )
        preprocessor = InteractionsPreprocessor()
        pairwise = PairwiseCorr(rectify=True, shuffle=False)
        spikes = preprocessor(loader())
        df_affinity = (
            pairwise.fit(spikes)
            .get_adjacency_df()
            .dropna(axis=1, thresh=5)
            .dropna(axis=0, thresh=5)
        )
        G = df_to_graph(df_affinity, rename_nodes=True)

        graph_stats = (
            graph_attrs
            .get_graph_attributes(G)
            .assign(session_name=session_name, block=block, bin_width=bin_width)
        )
        node_stats = (
            node_attrs
            .get_node_attributes(G, node_name="neuron_id")
            .assign(session_name=session_name, block=block,  bin_width=bin_width)
        )
        edge_stats = (
            graph_transformer
            .graph_to_edge_df(G)
            .assign(session_name=session_name, block=block, bin_width=bin_width)
        )
        return graph_stats, node_stats, edge_stats


In [3]:
neuron_types = load_derived_generic("neuron_types.csv").loc[
    lambda x: ~x["neuron_type"].isna()
]
sessions = (
    neuron_types.query("experiment_name == 'HAMILTON' and group in ('SAL', 'CIT')")[
        "session_name"
    ]
    .dropna()
    .unique()
)

df_distance = load_distances()
df_ensembles = (
    load_derived_generic("ensembles/fs - ensembles - true.csv")
    .drop_duplicates()
    .loc[lambda x: x.neuron_id.isin(neuron_types.neuron_id.unique())]
)
df_ensembles["in_ensemble"] = np.where(df_ensembles["ensemble_id"] != "-1", True, False)

df_responders = load_responders()

graph_attrs = GraphAttributes()
node_attrs = NodeAttributes()

graph_dfs = []
node_dfs = []
edge_dfs = []

for bin_width in tqdm((0.05, 0.1, 0.5, 1)):
    for block in ("pre", "base_shock"):
        g_transformer = GraphTransformer(
            relabel_nodes=True,
            weight_attr="weight",
            neuron_types=neuron_types,
            df_distance=df_distance,
            df_ensemble=df_ensembles.loc[
                (df_ensembles["block"] == block)
                & (df_ensembles["bin_width"] == bin_width)
            ],
        )
        for session_name in sessions:
            graph_stats, node_stats, edge_stats = get_graph_stats(
                session_name, bin_width, graph_transformer=g_transformer, block=block
            )
            graph_dfs.append(graph_stats)
            node_dfs.append(node_stats)
            edge_dfs.append(edge_stats)

df_graph = (
    pd.concat(graph_dfs)
    .reset_index(drop=True)
    .merge(
        neuron_types[["session_name", "group"]].drop_duplicates(),
    )
)
df_edge = (
    pd.concat(edge_dfs)
    .reset_index(drop=True)
    .merge(
        neuron_types[["session_name", "group"]].drop_duplicates(),
    )
)
df_node = (
    pd.concat(node_dfs)
    .reset_index(drop=True)
    .merge(neuron_types[["neuron_id","neuron_type", "group"]].drop_duplicates())
)
df_node = df_node.merge(
    df_ensembles[
        [
            "neuron_id",
            "in_ensemble",
            "block",
            "bin_width",
        ]
    ],
    how="left",
).drop_duplicates()
df_node = df_node.merge(df_responders)


100%|██████████| 4/4 [02:25<00:00, 36.35s/it]


In [17]:
dd = Config.derived_data_dir / "graph"

dd.mkdir(exist_ok=True)

df_graph.to_parquet(dd / "fs - graph.parquet", index=False)
df_node.to_parquet(dd / "fs - node.parquet", index=False)
df_edge.to_parquet(dd / "fs - edge.parquet", index=False)
df_responders.to_parquet(dd / "fs - responders.parquet", index=False)
